# UAVSAR Line of Sight

Brian Hawkins, Jet Propulsion Laboratory, California Institute of Technology

This notebook demonstrates how to compute the line-of-sight vector for each pixel of a UAVSAR RPI GRD product. Similar data can be downloaded from the UAVSAR webpage at

https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=SanAnd_08515_09015-000_17104-002_3079d_s01_L090_01#data

The crux of the problem is to determine the effective radar imaging time given a target location. This depends on the Doppler centroid used for processing, the technical details of which are described in the paper

> S. Hensley _et al._, "Residual motion estimation for UAVSAR: Implications of an electronically scanned array," 2009 IEEE Radar Conference, 2009, pp. 1-5, doi: 10.1109/RADAR.2009.4977065.

available for download [here](https://ieeexplore.ieee.org/document/4977065).  That paper and UAVSAR product definitions both make extensive use of the JPL "SCH" coordinate system described in the memo

> S. Hensley, "SCH Coordinates and Various Transformations", JPL Inter Office Memorandum, June 15, 2000.

In [1]:
import angler  # JPL proprietary
import logging as log
import matplotlib.pyplot as p
import numpy as np
from numpy import sin, cos, sqrt  # save a little typing
import os
import rdf  # JPL proprietary
import scipy.optimize as opt
import subprocess as sub
import sys

log.basicConfig(level=log.DEBUG, stream=sys.stdout)

In [2]:
os.chdir("los")
ann = rdf.open("SanAnd_08515_09015-000_17104-002_3079d_s01_L090HH_02.ann")

In [3]:
# Read radar geometry parameters from annotation file.
peglat = float(ann.get("Peg Latitude", "deg"))
peglon = float(ann.get("Peg Longitude", "deg"))
peghdg = float(ann.get("Peg Heading", "deg"))
coff = float(ann.get("Cross Track Offset From Reference Track", "m"))
alt = float(ann.get("Global Average Altitude", "m"))
ht = float(ann.get("Global Average Terrain Height", "m"))
yaw = np.deg2rad(float(ann.get("Global Average Yaw", "deg")))
pitch = np.deg2rad(float(ann.get("Global Average Pitch", "deg")))
esa = np.deg2rad(float(ann.get("Global Average ESA", "deg")))
smin = float(ann.get("Slant Range Data Starting Azimuth", "m"))
ds = float(ann.get("Slant Range Data Azimuth Spacing", "m"))
ns = int(ann.get("slt.set_rows"))
smax = smin + ds * ns

In [4]:
class RadarLocator:
    def __init__(self, coord: angler.SchCoordinates, coff=0.0, alt=12500.):
        """
        Radar reference track defined by constant cross-track (C)
        and vertical (H) offset from SCH peg point.
        """
        self.coord = coord
        self.c = coff
        self.h = alt
        
    def get_xyz(self, s):
        """Return radar ECEF XYZ position as a function of azimuth `s` coordinate (meters).
        """
        return self.coord.to_xyz((s, self.c, self.h))

coord = angler.SchCoordinates(peglat, peglon, peghdg)
radar = RadarLocator(coord, coff, alt)

In [5]:
# Read target geometry parameters from annotation file.
lat0 = float(ann.get("grd.row_addr", "deg"))
lon0 = float(ann.get("grd.col_addr", "deg"))
dlat = float(ann.get("grd.row_mult", "deg/pixel"))
dlon = float(ann.get("grd.col_mult", "deg/pixel"))
dem_fn = ann.get("DEM Used in Ground Projection")
nlon = int(ann.get("grd.set_cols"))
nlat = int(ann.get("grd.set_rows"))
# Load digital elevation model.
dem = np.memmap(dem_fn, dtype="f4", shape=(nlat, nlon), mode="r")

In [6]:
class TargetLocator:
    def __init__(self, lat0, lon0, dlat, dlon, dem=None):
        """
        GRD product geometry, parameters: lat/lon origin and spacing, all in degrees.
        Optional digital elevation model is 2D array with same dimensions as GRD product,
        contains height (in meters) for each pixel.
        """
        self.lat0 = lat0
        self.lon0 = lon0
        self.dlat = dlat
        self.dlon = dlon
        self.dem = dem
        
    def get_xyz(self, i, j):
        """Return target ECEF XYZ position as a function of GRD pixel row & column coordinates.
        """
        lat = self.lat0 + i * self.dlat
        lon = self.lon0 + j * self.dlon
        h = dem[i,j] if dem is not None else 0.0
        return angler.llh_to_xyz((lat, lon, h))

target = TargetLocator(lat0, lon0, dlat, dlon, dem)

In [7]:
def find_look_vector(target_xyz, radar: RadarLocator, yaw=0.0, pitch=0.0, esa=0.0,
                     smin=-200e3, smax=200e3, tol=1e-4):
    """
    Parameters
    ----------
    target_xyz : array_like
        Position of target in ECEF XYZ (meters)
    radar : RadarLocator
        Class describing UAVSAR reference trajectory
    yaw : float
        Yaw angle (radians)
    pitch : float
        Pitch angle (radians)
    esa : float
        Electronic steering angle (radians)
    smin : float
    smax : float
        Azimuth interval to search for radar position (meters)
    tol : float
        Tolerance for azimuth convergence (meters)
        
    Returns
    -------
    look_xyz : array_like
        Line of sight vector (radar-to-target, in meters).
    """
    target_xyz = np.asarray(target_xyz)
    # Find vector parallel to long axis of radar antenna, "nhat" in UAVSAR docs.
    # This is Equation (7) in Scott's paper.
    rot_yaw = angler.rotation_matrix(yaw, "z")
    rot_pitch = angler.rotation_matrix(pitch, "y")
    nhat_sch = rot_yaw.dot(rot_pitch).dot([1, 0, 0])
    # Compute look vector between target and radar given azimuth coordinate.
    def get_look_xyz(s):
        sch = [s, radar.c, radar.h]
        return target_xyz - radar.coord.to_xyz(sch)
    # Function of azimuth that's zero when target crosses beam center.
    def squint_error(s):
        look_xyz = get_look_xyz(s)
        unit_look_xyz = look_xyz / np.linalg.norm(look_xyz)
        # Need to rotate into local sch frame.
        sch = [s, radar.c, radar.h]
        unit_look_sch = radar.coord.local_cartesian_from_xyz(unit_look_xyz, origin_sch=sch)
        log.debug(f"look_sch = {np.array(unit_look_sch) * np.linalg.norm(look_xyz)}")
        # Equation (8) in Scott's paper
        return nhat_sch.dot(unit_look_sch) - np.sin(esa)
    # Find the root of above equation to get azimuth.
    sol = opt.root_scalar(squint_error, bracket=[smin, smax], xtol=tol)
    return get_look_xyz(sol.root)

In [8]:
# Demonstrate with target in the middle of the GRD product.
# Note that not all GRD pixels have radar data or valid geometry.
i, j = nlat // 2, nlon // 2
target_xyz = target.get_xyz(i, j)
target_llh = angler.xyz_to_llh(target_xyz)
target_llh

(33.47256647999999, -116.49204084000002, 1276.1429443361124)

In [9]:
look_xyz = find_look_vector(target_xyz, radar, yaw, pitch, esa)
look_xyz

DEBUG:root:look_sch = [197830.12031675  15598.44238152 -14303.88836043]
DEBUG:root:look_sch = [-202183.17902088   15598.44238152  -14440.29793992]
DEBUG:root:look_sch = [ -8235.66398132  15598.44238152 -11244.04843455]
DEBUG:root:look_sch = [ 87236.86494618  15598.44238152 -11834.65099691]
DEBUG:root:look_sch = [ 18517.16890679  15598.44238152 -11265.58568805]
DEBUG:root:look_sch = [  1308.63866702  15598.44238152 -11238.87172351]
DEBUG:root:look_sch = [  -123.01198736  15598.44238152 -11238.73881653]
DEBUG:root:look_sch = [   -16.60317672  15598.44238152 -11238.73765328]
DEBUG:root:look_sch = [   -16.7052778   15598.44238152 -11238.73765355]
DEBUG:root:look_sch = [   -16.70522779  15598.44238152 -11238.73765355]


array([ 6931.581941  , 16585.47232093,  6818.81129876])

In [10]:
enu2xyz = angler.enu2xyz_matrix(target_llh[0], target_llh[1])
look_enu = enu2xyz.transpose() @ look_xyz
look_enu

array([ -1194.59908734,  15580.25321349, -11200.42636087])

In [11]:
# Sanity check: up component should be roughly target height minus radar height.
# Not exact equality due to curvature of surface and reference track.
target_llh[2] - alt

-11219.599755663889

In [12]:
# The norm of the look vector is the range to the target.
ρ = np.linalg.norm(look_xyz)
ρ

19225.527504885365

## Compare with `make_geolook`

Scott Hensley (and contributors) wrote a Fortran program named `make_geolook` that implements these look vector and related calculations.  Below we'll configure its input file, run the program, and compare its output with the above.

In [13]:
# Update template config file with metadata from annotation file.
cfg = rdf.load("mgl_template.rdf")

In [14]:
cfg.set("Null Identification File", dem_fn)
cfg.set("Number of Samples and Lines for Scene", (nlon, nlat))
cfg.set("Corner Latitude and Longtitude for Scene", (lat0, lon0), "deg")
cfg.set("Latitude amd Longitude Pixel Spacings", (dlat, dlon), "deg")
cfg.set("Peg Latitude, Longtitude and Heading", (peglat, peglon, peghdg), "deg")
yawd, pitchd, esad = np.rad2deg([yaw, pitch, esa])
cfg.set("Global Averaged Yaw, Pitch and Azimuth Steering Angle", (yawd, pitchd, esad), "deg")
cfg.set("Global Averaged Platform Altitude", alt, "m")
cfg.set("Global Terrain Height for Scene", ht, "m")

cfg_fn = "mgl_config.rdf"
cfg.write(cfg_fn)

In [15]:
# Output files have fixed names but you can specify a prefix.
prefix = "mgl"
los_fn = f"{prefix}_enulos.dat"

if not os.path.exists(los_fn):
    sub.run(["make_geolook", cfg_fn, "1", prefix, dem_fn], check=True)
else:
    print("using cached make_geolook result")

using cached make_geolook result


In [16]:
los = np.memmap(los_fn, dtype="float64")
los.shape = nlat, nlon, 3
print("make_geolook ENU LOS (m) =", los[i,j])

make_geolook ENU LOS (m) = [ -1179.00989758  15581.43171718 -11200.42636919]


In [17]:
print("simple ENU LOS (m) =", look_enu)

simple ENU LOS (m) = [ -1194.59908734  15580.25321349 -11200.42636087]


In [18]:
platform_xyz = target_xyz - look_xyz
platform_sch = radar.coord.from_xyz(platform_xyz)
xyz2sch = radar.coord.local_cartesian_matrix(platform_sch)
dlook_sch = xyz2sch @ enu2xyz @ (los[i,j] - look_enu)
for k, ax in enumerate("sch"):
    print(f"LOS difference {ax} (m) =", dlook_sch[k])

LOS difference s (m) = 15.633672251318385
LOS difference c (m) = 1.7187097085479471e-06
LOS difference h (m) = 1.910407010093331e-05


In [19]:
u1 = los[i,j] / np.linalg.norm(los[i,j])
u2 = look_enu / np.linalg.norm(look_enu)
angle = np.arccos(u1 @ u2)
print("angular difference (deg) =", np.rad2deg(angle))

angular difference (deg) = 0.04659136324779652


## Reconcile

The two implementations produce the same look direction to within a small fraction of a degree.  They would be practically equivalent for most applications that want to project a 3D field onto the radar line of sight.

There is a difference of a few meters in the along-track direction.  This would be relevant for applications that would like to resample between map and radar coordinates.  The offset is due to the fact that the Doppler depends on the target height, but the UAVSAR processor performs a reskew operation using a constant reference height.  The error therefore depends on this sensitivity and the difference between the actual target height (DEM) and the height assumed for reskew.  The `make_geolook` program accounts for this distortion, and a formula is provided below.

In [20]:
dh = ht - dem[i,j]
print("DEM height (m) =", dem[i,j])
print("reskew reference height (m) =", ht)
print("difference (m) =", dh)

DEM height (m) = 1276.143
reskew reference height (m) = 485.246273
difference (m) = -790.8966713359375


In [21]:
def dls_dht(alt, ht, yaw, pitch, esa, ρ, a=6378e3):
    """Derivative of s-component of look vector with respect to terrain height.
    """
    rp = a + alt
    rt = a + ht
    cosθ = (rp**2 + ρ**2 - rt**2) / (2 * rp * ρ)
    cosβ = (cosθ + sin(pitch) * sin(esa)) / (cos(pitch) * cos(esa))
    sinβ = sqrt(1 - cosβ**2)  # left looking
    x1 = (sin(yaw) * cosβ - sin(pitch) * cos(yaw) * sinβ) / (cos(pitch) * sinβ)
    return x1 * (rt / rp) / ρ

a = radar.coord.radius
ds = dls_dht(alt, ht, yaw, pitch, esa, ρ, a=a) * dh * ρ
ds

15.043665031909626

---
© 2021 California Institute of Technology. Government sponsorship acknowledged.

The research was carried out at the Jet Propulsion Laboratory, California Institute of Technology, under a contract with the National Aeronautics and Space Administration (80NM0018D0004)